In [ ]:
import csv
import time
import random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict

## Graph generation with 1000 and 10000 nodes. 
### The graph depth for the 1000 node graph is taken between 0.005 and 0.1, and that for the 10,000 node graph is between 0.0005 and 0.01. 

In [ ]:
num_nodes = 10000    # Set the number of nodes 
G = nx.DiGraph()     # Creating a directed graph

G.add_nodes_from(range(1, num_nodes + 1))

# Adding random directed edges with random weights
count = 0
for i in range(num_nodes):
    for j in range(num_nodes):
        if i != j and random.random() < 0.01:     # depth
            weight = random.randint(1, 10)
            G.add_edge(i + 1, j + 1, weight = weight)
            count += 1

# Ensuring weak connectivity if not already so  
extra_edges = 0
while not nx.is_weakly_connected(G):
    # Adding edges to connect components
    node1 = random.randint(1, num_nodes)
    node2 = random.randint(1, num_nodes)
    weight = random.randint(1, 10)
    G.add_edge(node1, node2, weight = weight)
    extra_edges += 1

print("Graph is now weakly connected.")
print('Extra edges added:', extra_edges)
print('Number of connections:', count + extra_edges)
print('Number of connections:', count)

### Visualize a subgraph  

In [ ]:
subgraph_nodes = random.sample(G.nodes(), 100)  # Set the no. of random nodes for visualization
subgraph = G.subgraph(subgraph_nodes)

pos = nx.spring_layout(subgraph, seed = 42)  

plt.figure(figsize = (12, 10))
nx.draw(subgraph, pos,
        with_labels = True,
        font_weight = 'bold',
        node_size = 300,
        node_color = 'skyblue',
        font_size = 8,
        edge_color = 'gray',
        linewidths = 0.5)

plt.show()

### CSV export  

In [ ]:
# Creating csv file for the graph

csv_file_path = "10knodes_1e-2.csv"    # Change it for every graph configuration

with open(csv_file_path, 'w', newline = '') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Source', 'Target', 'Weight'])
    for edge in G.edges(data = True):
        csv_writer.writerow([edge[0], edge[1], edge[2]['weight']])
print(f"CSV file saved at: {csv_file_path}")

In [ ]:
# Check for orphan nodes!
orphans = 0
for node in G.nodes():
    incoming_edges = G.in_degree(node)
    outgoing_edges = G.out_degree(node)
    if incoming_edges == 0 and outgoing_edges == 0:
        orphans += 1
print('Number of orphan nodes:', orphans)

In [ ]:
# Read the graph csv 
df = pd.read_csv(csv_file_path)

In [ ]:
# Convert node labels to strings
df['Source'] = df['Source'].astype(str)
df['Target'] = df['Target'].astype(str)

In [ ]:
df.head()

## Dijkstra's algorithm implementation from scratch
### For average runtime across `100` traversals with `random` input in each traversal

In [ ]:
# Creating a class for graph structure

class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = defaultdict(list)
        self.distances = {}

    def addNode(self, value):
        self.nodes.add(value)

    def addEdge(self, fromNode, toNode, distance):
        self.edges[fromNode].append(toNode)
        self.distances[(fromNode, toNode)] = distance
 
     
# Main algorithm

def dijkstra(graph, initial):
    visited = defaultdict(lambda: float('inf'))
    path = defaultdict(list)
    nodes = set(graph.nodes)

    visited[initial] = 0

    while nodes:
        minNode = min(nodes, key=lambda node: visited[node])
        nodes.remove(minNode)
        currentWeight = visited[minNode]

        for edge in graph.edges[minNode]:
            if (minNode, edge) not in graph.distances:
                continue

            weight = currentWeight + graph.distances[(minNode, edge)]
            if weight < visited[edge]:
                visited[edge] = weight
                path[edge] = path[minNode] + [minNode]

    return visited, path

# Run the algorithm for specific configuration

def main():
    graph = Graph()

    unique_nodes = set(df['Source'].astype(str).unique()) | set(df['Target'].astype(str).unique())

    for node in unique_nodes:
        graph.addNode(node)

    for index, row in df.iterrows():
        graph.addEdge(row['Source'], row['Target'], row['Weight'])

    total_time = 0

    for _ in range(100):
        start_nodes = random.sample(unique_nodes, 1)[0]
        end_nodes = random.sample(unique_nodes, 1)[0]

        start = time.time()
        visited, path = dijkstra(graph, start_nodes)
        if end_nodes in visited:
            optimal_path = path[end_nodes] + [end_nodes]
            optimal_distance = visited[end_nodes]
            print(f"Shortest Path from {start_nodes} to {end_nodes}: {optimal_path}")
            print(f"Optimal Distance: {optimal_distance}")
        else:
            print(f"No path found from {start_nodes} to {end_nodes}")
        end = time.time()
        elapsed_time = round((end - start), 4)
        print('Time elapsed:', elapsed_time)
        total_time += elapsed_time

    average_time = total_time / 100
    print("")
    print(f'AVERAGE TIME AFTER 100 TRAVERSALS: {round(average_time, 4)}s/ traversal')

if __name__ == "__main__":
    main()

## Implementation using `shortest_path()` method `from networkx`
### For average runtime across `100` traversals with `random` input in each traversal

In [ ]:
def main():
    total_time = 0

    for _ in range(100):
        start_nodes, end_nodes = random.sample(G.nodes, 2)

        start = time.time()
        try:
            optimal_path = nx.shortest_path(G, source=start_nodes, target=end_nodes, weight='Weight') # Default method is Dijkstra's
            optimal_distance = nx.shortest_path_length(G, source=start_nodes, target=end_nodes, weight='Weight')
            print(f"Shortest Path from {start_nodes} to {end_nodes}: {optimal_path}")
            print(f"Optimal Distance: {optimal_distance}")
        except nx.NetworkXNoPath:
            print(f"No path found from {start_nodes} to {end_nodes}")
        end = time.time()
        elapsed_time = round((end - start), 4)
        print('Time elapsed:', elapsed_time)
        total_time += elapsed_time

    average_time = total_time / 100
    print("")
    print(f'AVERAGE TIME AFTER 100 TRAVERSALS: {round(average_time, 4)}s/ traversal')

if __name__ == "__main__":
    main()